In [1]:
import pandas as pd 
import math
import variables
import random

In [2]:
def generate_chol(profiles):
    for index,row in profiles.iterrows():
        cholesterol_value = row['cholesterol']
        if cholesterol_value == 1:
            profiles.loc[index, 'cholesterol'] = random.randint(50,200)
        elif cholesterol_value == 2:
            profiles.loc[index, 'cholesterol'] = random.randint(201,240)
        else:
            profiles.loc[index, 'cholesterol'] = random.randint(241,500)
    
    return profiles

In [7]:
df = pd.read_csv('cardio.csv', sep=';')

cleaned_df = df[df['ap_hi'] < 250]  
cleaned_df = cleaned_df[cleaned_df['ap_lo'] < 250] 
cleaned_df = generate_chol(cleaned_df)

In [8]:
def calculo(profiles):
    for index,row in profiles.iterrows():
        gender = row['gender']
        age = int(row['age'] / 365)
        cholesterol = row['cholesterol']
        systolic = row['ap_hi']
        smoker = row['smoke']
        
        # Step 1 - CHD
        if gender == 1:
            internal = math.exp(variables.ALPHA_CHD_WOMEN)
            external = math.exp(-internal)
            s0 = external * (age - 20) * variables.P_CHD_WOMEN
            s0_10 = external * (age - 10) * variables.P_CHD_WOMEN
        else:
            internal = math.exp(variables.ALPHA_CHD_MEN)
            external = math.exp(-internal)
            s0 = external * (age - 20) * variables.P_CHD_MEN
            s0_10 = external * (age - 10) * variables.P_CHD_MEN
        
        # Step 2 - CHD
        w = (variables.CHOLESTEROL_CHD * (cholesterol - 6)) + (variables.SYSTOLIC_CHD * (systolic - 120)) + (variables.SMOKER_CHD * smoker)
        
        # Step 3 - CHD
        w_value = math.exp(w)
        s0 = s0 * w_value
        s0_10 = s0_10 * w_value
        
        # Step 4 - CHD
        if s0 != 0:
            s10 = s0_10 / s0
        else:
            print(s0)
            s10 = s0_10 
        
        # Step 5 - CHD
        risk = 1 - s10
        
        # Step 1 - NON CHD
        if gender == 1:
            internal = math.exp(variables.ALPHA_NONCHD_WOMEN)
            external = math.exp(-internal)
            s0 = external * (age - 20) * variables.P_NONCHD_WOMEN
            s0_10 = external * (age - 10) * variables.P_NONCHD_WOMEN
        else:
            internal = math.exp(variables.ALPHA_NONCHD_MEN)
            external = math.exp(-internal)
            s0 = external * (age - 20) * variables.P_NONCHD_MEN
            s0_10 = external * (age - 10) * variables.P_NONCHD_MEN
        
        # Step 2 - NON CHD
        w = (variables.CHOLESTEROL_NONCHD * (cholesterol - 6)) + (variables.SYSTOLIC_NONCHD * (systolic - 120)) + (variables.SMOKER_NONCHD * smoker)
        
        # Step 3 - NON CHD
        w_value = math.exp(w)
        s0 = s0 * w_value
        s0_10 = s0_10 * w_value
        
        # Step 4 - NON CHD
        if s0 != 0:
            s10 = s0_10 / s0
        else:
            print(s0)
            s10 = s0_10 
        
        # Step 5 - NON CHD
        risk_nonchd = 1 - s10
            
        # Final value
        profiles.loc[index, 'score'] = risk - risk_nonchd
    
    return profiles
        

In [12]:
final = calculo(cleaned_df)
print(final)
print(f"media: {final['score'].mean()}")
print(f"maximo: {final['score'].max()}")
print(f"minimo: {final['score'].min()}")

          id    age  gender  height  weight  ap_hi  ap_lo  cholesterol  gluc  \
0          0  18393       2     168    62.0    110     80          110     1   
1          1  20228       1     156    85.0    140     90          299     1   
2          2  18857       1     165    64.0    130     70          312     1   
3          3  17623       2     169    82.0    150    100           68     1   
4          4  17474       1     156    56.0    100     60           87     1   
...      ...    ...     ...     ...     ...    ...    ...          ...   ...   
69995  99993  19240       2     168    76.0    120     80          181     1   
69996  99995  22601       1     158   126.0    140     90          231     2   
69997  99996  19066       2     183   105.0    180     90          341     1   
69998  99998  22431       1     163    72.0    135     80          162     2   
69999  99999  20540       1     170    72.0    120     80          205     1   

       smoke  alco  active  cardio     